Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

On Monday we talked about summarizing your documents using just token counts. Today, we're going to learn about a much more sophisticated approach - learning 'topics' from documents. Topics are a latent structure. They are not directly observable in the data, but we know they're there by reading them.

> **latent**: existing but not yet developed or manifest; hidden or concealed.

## Use Cases
Primary use case: what the hell are your documents about? Who might want to know that in industry - 
* Identifying common themes in customer reviews
* Discovering the needle in a haystack 
* Monitoring communications (Email - State Department) 

## Learning Objectives
*At the end of the lesson you should be able to:*
* <a href="#p1">Part 1</a>: Describe how an LDA Model works
* <a href="#p2">Part 2</a>: Estimate a LDA Model with Gensim
* <a href="#p3">Part 3</a>: Interpret LDA results
* <a href="#p4">Part 4</a>: Select the appropriate number of topics


# Latent Dirchilet Allocation Models (Learn)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a model available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Literary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for literary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelty of British women publishing novels during the early 19th century. 

Can we help close a literary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning algorithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [1]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [2]:
gensim.__version__

'3.8.1'

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chunks

In [3]:
path = './data/austen-brontë-split'

### Text Preprocessing
**Activity**: update the function `tokenize` with any technique you have learned so far this week. 

In [4]:
# 1) Plain Python - ''.split command
# 2) Spacy - just the lemmas from the document
# 3) Gensim - simple_preprocess

STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [14]:
ex_str = ('This is an example of a string.  ')

In [7]:
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

CUSTOM_FILTERS = [lambda x: x.lower(),         # Lowercase
                  strip_multiple_whitespaces,  # Remove repeating white space
                  strip_numeric,               # Remove numbers
                  remove_stopwords,            # Remove stopwords
                  strip_non_alphanum,          # Remove non-alpha numeric characters,
                  strip_short                  # Remove words less than 3 characters long
                  ]

# Preprocess text with custom filters
preprocess_string(ex_str, CUSTOM_FILTERS)

['example', 'string']

In [8]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [9]:
tokens = gather_data(path)

In [10]:
tokens[0][0:10]

['liberty',
 'society',
 'amusement',
 'till',
 'father',
 'point',
 'gained',
 'depended',
 'fortitude',
 'far']

In [11]:
"this is a sample string with a \n newline character".replace('\n', '')

'this is a sample string with a  newline character'

In [15]:
' '.join(ex_str.split())

'This is an example of a string.'

## Follow Along

In [16]:
titles = [t[:-4] for t in os.listdir(path) if os.path.isdir(t) == False]

In [17]:
len(titles)

813

In [18]:
len(tokens)

813

### Author DataFrame


In [19]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [20]:
df.head()

,tokens
Austen_Sense0057,"[liberty, society, amusement, till, father, po..."
Austen_Pride0013,"[attention, messages, sister, hurst, bingley, ..."
CBronte_Villette0007,"[likes, better, turns, away, thoughts, consoli..."
Austen_Pride0109,"[attentions, spring, score, let, sit, understa..."
Austen_Emma0123,"[far, strongly, issue, feeling, softer, uprigh..."


In [21]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [22]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [23]:
df.author.value_counts()

0    441
1    372
Name: author, dtype: int64

In [24]:
df.head()

,tokens,author,book,section
Austen_Sense0057,"[liberty, society, amusement, till, father, po...",1,Sense,57
Austen_Pride0013,"[attention, messages, sister, hurst, bingley, ...",1,Pride,13
CBronte_Villette0007,"[likes, better, turns, away, thoughts, consoli...",0,Villette,7
Austen_Pride0109,"[attentions, spring, score, let, sit, understa...",1,Pride,109
Austen_Emma0123,"[far, strongly, issue, feeling, softer, uprigh...",1,Emma,123


### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our function. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this function later on. 

In [25]:
def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(text)
                yield tokens

In [26]:
streaming_data = doc_stream(path)

In [27]:
type(streaming_data)

generator

In [28]:
# gather_data => returns a list
# doc_stream => returns a generator

In [31]:
next(streaming_data) # Returns one document at a time from the generator

['likes',
 'better',
 'turns',
 'away',
 'thoughts',
 'consoling',
 'improving',
 'occasion',
 'inculcating',
 'maxims',
 'philosophy',
 'whereof',
 'tolerable',
 'stock',
 'ready',
 'application',
 'stopped',
 'putting',
 'fingers',
 'ears',
 'words',
 'uttered',
 'lying',
 'mat',
 'face',
 'flags',
 'warren',
 'cook',
 'root',
 'position',
 'allowed',
 'lie',
 'till',
 'chose',
 'rise',
 'accord',
 'graham',
 'forgot',
 'impatience',
 'evening',
 'accosted',
 'usual',
 'friends',
 'gone',
 'wrenched',
 'hand',
 'eye',
 'flashed',
 'bid',
 'good',
 'night',
 'look',
 'face',
 'day',
 'treated',
 'indifference',
 'grew',
 'like',
 'bit',
 'marble',
 'day',
 'teased',
 'know',
 'matter',
 'lips',
 'unclose',
 'course',
 'feel',
 'real',
 'anger',
 'match',
 'unequal',
 'way',
 'tried',
 'soothing',
 'coaxing',
 'angry',
 'tears',
 'answered',
 'petted',
 'friends',
 'incidents',
 'lost',
 'remarked',
 'rebuff',
 'seek',
 'follow',
 'way',
 'solicit',
 'notice',
 'told',
 'carry',
 'book

### Gensim LDA Topic Modeling

In [32]:
# A Dictionary Representation of all the words in our corpus
id2word = corpora.Dictionary(doc_stream(path))

In [33]:
id2word.token2id['england']

78

In [34]:
id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

[(78, 3), (353, 1), (6207, 1), (18936, 1)]

In [35]:
import sys
print(sys.getsizeof(id2word))
print(sys.getsizeof(tokens))

56
7056


In [36]:
len(id2word.keys())

22095

In [37]:
# Let's remove extreme values from the dataset
id2word.filter_extremes(no_below=5, no_above=0.95)

In [38]:
len(id2word.keys())

8102

In [39]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here
# Although abstracted away - tokenization IS happening in the doc_stream f(x)
corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [41]:
corpus[345][:10]

[(4, 1),
 (17, 1),
 (28, 1),
 (30, 1),
 (31, 2),
 (40, 1),
 (46, 1),
 (51, 1),
 (55, 1),
 (71, 1)]

In [40]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [41]:
lda.print_topics()

[(0,
  '0.013*"miss" + 0.005*"bingley" + 0.004*"ingram" + 0.004*"elizabeth" + 0.004*"english" + 0.004*"monsieur" + 0.004*"good" + 0.004*"little" + 0.004*"darcy" + 0.004*"books"'),
 (1,
  '0.008*"reed" + 0.004*"thought" + 0.004*"thing" + 0.004*"vault" + 0.003*"like" + 0.003*"love" + 0.003*"mother" + 0.003*"strange" + 0.003*"john" + 0.003*"child"'),
 (2,
  '0.011*"weston" + 0.010*"churchill" + 0.008*"frank" + 0.007*"miss" + 0.006*"elton" + 0.005*"life" + 0.005*"great" + 0.005*"thing" + 0.005*"shall" + 0.004*"like"'),
 (3,
  '0.009*"hawkins" + 0.006*"news" + 0.005*"miss" + 0.004*"says" + 0.004*"campbell" + 0.004*"bates" + 0.003*"mother" + 0.003*"precise" + 0.003*"worlds" + 0.003*"deaf"'),
 (4,
  '0.015*"dear" + 0.009*"perry" + 0.009*"sea" + 0.008*"little" + 0.007*"children" + 0.007*"good" + 0.006*"gruel" + 0.006*"air" + 0.006*"south" + 0.006*"sir"'),
 (5,
  '0.005*"little" + 0.005*"like" + 0.004*"thought" + 0.004*"time" + 0.003*"know" + 0.003*"hand" + 0.003*"long" + 0.003*"day" + 0.003*"g

In [42]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [43]:
topics = [' '.join(t[0:5]) for t in words]

In [45]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
pounds mother sure think year

------ Topic 1 ------
pelet thought like little man

------ Topic 2 ------
madame little like day vous

------ Topic 3 ------
night look like old est

------ Topic 4 ------
little madame like thought know

------ Topic 5 ------
elizabeth darcy miss bingley bennet

------ Topic 6 ------
know jane life like long

------ Topic 7 ------
frances monsieur hunsden little english

------ Topic 8 ------
good little like emma elton

------ Topic 9 ------
sir long jane rochester little

------ Topic 10 ------
like little rochester miss room

------ Topic 11 ------
emma miss harriet thing weston

------ Topic 12 ------
jane emma little thing miss

------ Topic 13 ------
elinor marianne sister mother time

------ Topic 14 ------
monsieur henri georgiana mdlle little



## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [44]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [45]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.035837 -0.087028       1        1  19.998714
13     0.128797  0.047815       2        1  17.269522
5     -0.018748 -0.066216       3        1  16.610149
10     0.110067  0.041641       4        1  15.069300
11     0.136440  0.052373       5        1  12.892591
8     -0.004402 -0.090995       6        1   6.455765
14    -0.018052 -0.107779       7        1   5.041890
6      0.079492 -0.051574       8        1   3.587849
12    -0.006304 -0.026825       9        1   0.991836
0      0.009417 -0.023443      10        1   0.728118
2      0.045614  0.083109      11        1   0.692152
9     -0.090031  0.004590      12        1   0.319363
1     -0.144117  0.057869      13        1   0.157182
4     -0.078693  0.098340      14        1   0.147970
3     -0.113644  0.068122      15        1   0.037594, topic_info=           Term         Freq        Total Category  logprob  loglift
165        miss  1507.000000  1507.000000  Default  30.0000  30.0000
148      little  1720.000000  1720.000000  Default  29.0000  29.0000
948        emma   885.000000   885.000000  Default  28.0000  28.0000
1089       dear   582.000000   582.000000  Default  27.0000  27.0000
405        like  1332.000000  1332.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
987   knightley     0.234649   402.497620  Topic15  -6.2189   0.4387
2056    colonel     0.225185   300.780182  Topic15  -6.2601   0.6889
148      little     0.253536  1720.167236  Topic15  -6.1415  -0.9363
984        jane     0.237802   913.394348  Topic15  -6.2056  -0.3674
948        emma     0.188841   885.744385  Topic15  -6.4361  -0.5672

[1092 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1946      2  0.030231  abbey
1946      5  0.906924  abbey
1946      8  0.030231  abbey
6203      1  0.121992  abbot
6203      8  0.853943  abbot
...     ...       ...    ...
292      11  0.006026  young
292      12  0.003013  young
6585      3  0.235098  zelie
6585      6  0.587744  zelie
6585      7  0.117549  zelie

[3879 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 14, 6, 11, 12, 9, 15, 7, 13, 1, 3, 10, 2, 5, 4])

### Overall Model / Documents

In [46]:
lda[corpus[0]]

[(10, 0.99742657)]

In [47]:
distro = [lda[d] for d in corpus]

In [49]:
distro[1]

[(10, 0.050196726), (13, 0.94735783)]

In [50]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [51]:
len(new_distro)

813

In [52]:
df.head()

,tokens,author,book,section
Austen_Sense0057,"[liberty, society, amusement, till, father, po...",1,Sense,57
Austen_Pride0013,"[attention, messages, sister, hurst, bingley, ...",1,Pride,13
CBronte_Villette0007,"[likes, better, turns, away, thoughts, consoli...",0,Villette,7
Austen_Pride0109,"[attentions, spring, score, let, sit, understa...",1,Pride,109
Austen_Emma0123,"[far, strongly, issue, feeling, softer, uprigh...",1,Emma,123


In [53]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [54]:
df.head()

,miss bingley ingram elizabeth english,reed thought thing vault like,weston churchill frank miss elton,hawkins news miss says campbell,dear perry sea little children,little like thought time know,miss little papa harriet bessie,like little night madame thought,miss little like think thought,like little likeness man yes,elinor marianne sister mother edward,emma harriet miss knightley thing,know like jane miss day,elizabeth jane miss darcy know,rochester little sir good day,author
Austen_Sense0057,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.997427,0.000000,0.0,0.000000,0.0,Austen
Austen_Pride0013,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.050193,0.000000,0.0,0.947362,0.0,Austen
CBronte_Villette0007,0.0,0.0,0.0,0.0,0.0,0.290945,0.36969,0.337258,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,CBronte
Austen_Pride0109,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.281097,0.000000,0.0,0.716396,0.0,Austen
Austen_Emma0123,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.055417,0.584784,0.0,0.357777,0.0,Austen


In [55]:
df.groupby('author').mean()

,miss bingley ingram elizabeth english,reed thought thing vault like,weston churchill frank miss elton,hawkins news miss says campbell,dear perry sea little children,little like thought time know,miss little papa harriet bessie,like little night madame thought,miss little like think thought,like little likeness man yes,elinor marianne sister mother edward,emma harriet miss knightley thing,know like jane miss day,elizabeth jane miss darcy know,rochester little sir good day
author,,,,,,,,,,,,,,,
Austen,0.005276,0.000000,0.011047,0.000438,0.00295,0.000888,0.014403,0.006993,0.002147,0.000890,0.295458,0.275726,0.000493,0.380713,0.000000
CBronte,0.009048,0.002605,0.002990,0.000000,0.00000,0.301037,0.053180,0.346904,0.113389,0.004525,0.032121,0.005557,0.018340,0.017344,0.090535


## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text preprocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [87]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus,
                                                        start=2, 
                                                        limit=40, 
                                                        step=2,
                                                        passes=100)

In [89]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)